# Deep Reinforcement Learning

<div class="not_clean">

Cilj **deep reinforcement learning** je najti optimalne korake za pridobitev največje možne nagrade v danem okolju.
    
Zadevo je najlažje razložiti na primeru robota, ki igra šah (oziroma neko drugo igro). V tem primeru imamo:
* **agent** - "oseba" oziroma entiteta, ki izvaja akcije
* **action** - možne akcije, ki jih agent lahko izvede. Na primer premik figure, itd
* **enviornment** - okoje v katerem agent deluje. V našem primeru je to šahovska plošča. Znotraj okolja imamo informacije o pozicijah različnih figure, kako se različne figure premikajo, itd.
    
Cilj agenta je zmagati igro šaha. Agent sedaj premika figure. Za vsak premik dobi informacijo o tem ali je bila poteza dobre ali slaba. Če je zajel nasprotnikovo figuro, je to bila dobra poteza in agent bi prejel pozitiven **reward**. Če je agent izgubil figuro, bi to bila slaba poteza in bi prejel negativn **reward**. Obstajajo pa tudi poteze, ki ne zajamejo figur. Ampak so del naše taktike s katero želimo v prihodnjih potezah zajeti figure. Cilj našega modela je, da se nauči optimalne strategije s katero pridobi največjo možno nagrado.
    
Ena izmed razlik med Reinforcement Learning in ostalimi machine learning algoritmi je, da imamo pri reinforcement learning-u zakasnele nagrade. To pomeni, da v določenih primerih agent izvede večje število akcij predno dobi katerokoli nagrado. Se pravi, da pri odločanju agent potrebuje upoštevati tudi možne prihodnje nagrade, ne samo trenutno posledice.
</div>

<div class="not_clean">

Reinforcement learning se uporablja v industrijah kot so:
* Self-driving vehicles, kjer se lahko avto nauči planiranje poti, parkiranje, prehitevanje, itd.
* Google DeepMind je uporabil reinforcement learning za optimizacijo hlajenja data centrov, kar je privedlo do 40% zmanjšanja stroškov. AI vsakih 5min naredi snapshot stanja data centra, AI se nato odloči za določeno akcijo, ki bo privedla do optimalne porabe. [Google DeepMind RL cooling](https://www.deepmind.com/blog/safety-first-ai-for-autonomous-data-centre-cooling-and-industrial-control)  
* trading and finances, kjer se RL model lahko odloči za določeno akcijo (buy or sell)
* robotika [Učenje robota metati](https://www.youtube.com/watch?v=-O-E1nFm6-A)
* natural language processing, 
* marketing and advertisement, ...
</div>

<div class="not_clean">

Uporabljajo se tudi v gaming industriji sam nam igre dajejo perfekten playing-ground. 
    
Igre nudijo obstoječa kompleksna okolja, ki niso preobsežna. Če želimo, da naš robot pobere objekt v realnem svetu more sprocesirati ogromne količine senzoričnih podatkov.
    
Zanimiv primer je prišel od OpenAI, ki so ustvarili profesionalne bot-e za igro Dota 2 - [LINK](https://www.youtube.com/watch?v=UZHTNBMAfAA). Boti so bili tako dobro, da so premagali takratke World Champions.
</div>

---

# Taxi game

<div class="not_clean">

Poglejmo si osnovne principe na primeru igre.
    
Simulirali bomo **self-driving taxi**. Cilj taxija, da pobere potnika na eni lokaciji in ga odloži na drugi lokaciji. Za naš taxi želimo, da:
* odloži potnika na pravilni lokaciji
* izbere najkrajšo pot

</div>

[Taxi_game documentation](https://gymnasium.farama.org/environments/toy_text/taxi/#taxi)

<div>
<img src="images/taxi_game.png" width="500"/>
</div>

## State space

<div class="not_clean">

Pri RL se agent znajde v nekem stanju (**state**) in opravi **action**. Ta action ga premakne v naslednji **state**. Znotraj **stata** bi morali najti informacije na podlagi katerih se agent odloči za določeno akcijo - pozicija taxija, pozicija potnika, itd.
    
**State space** je set vseh možnih situacij v katerih bi se lahko naš taxi znašel.
    
Naše polje imamo razdeljeno na **5**x**5** grid, kar nam da 25 različnih lokacij v katerih se taxi lahko nahaja. Lokacija na sliki je (3, 1).
    
Poleg teh lokacij imamo **4** različne pozicije na katerih taxi pobere oziroma dostavi ljudi
* R - (0, 0)
* G - (0, 4)
* Y - (4, 0)
* B - (4, 3)
Naš potnik se nahaja na lokaciji R (0, 0) in želi priti na Z (0, 4).
    
Dodatno moramo upoštevati stanje potinka, ki se lahko nahaja ali na izmed **4-ih** postaj ali **v** taxiju. Tako imamo dodatnih **5** stanj za potnika.
    
Skupaj imamo 5 * 5 * 4 * 5 = 500 vseh možnih stanj.
</div>

## Action space

<div class="not_clean">

Taxi (oziroma agent) se bo znašel v nekem stanju in se odločil za določeno akcijo. Na voljo ima 6 možnosti:
* dol
* gor
* desno
* levo
* pickup
* dropoff
    
To je naš action space - nabor vseh akcij katere lahko opravimo.
</div>

## Rewards

<div class="not_clean">

Ko taxi opravi določeno akcijo je premaknjen v novo stanje. Poleg tega prejme določen **reward**. Njegov cilj je maximizirat dosežene nagrade.
    
Mi mu želimo dati visoko nagrado za uspešno dostavljenega potnika. Če poizkusi odložiti potinka na napačni lokaciji mu želimo dati visok negativen reward (oziroma kazen). Dodatno mu želimo dodeliti majhno negatvno kazen za vsak korak katerega opravi. S tem ga želimo prisiliti, da opravi najkrajšo pot.

Dodatno mu bomo dodelili majhno negativno nagrado, če bo taxi želel prečkati steno.
</div>

## Implementacija v Gymnasium

<div class="not_clean">

Naše okolje lahko najdemo znotraj python knjižnjice Gymnasium - https://gymnasium.farama.org/
Knjižnjica je še nedolgo nazaj obstajala kot knjižnjica OpenAI Gym.
    
Gymnasium nam nudi API wrapper za veliko različnih iger.
</div>

`$ pip install gymnasium`

oziroma

`$ pip install gymnasium[toy-text]`

Dodatno bomo potrebovali:

```
gymnasium[toy-text]
matplotlib
tensorflow
stable_baselines3[extra]>=2.0.0a9
gym
```

in še pytorch (inštalacija odvisna od OS in našega račnalnika).
`$ pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117`

<div class="not_clean">

Glavni interface s katerim delamo je `env` in predstavlja naše okolje.
    
* Z `env.reset()` metodo resetiramo okolje in ga postavimo v začetni položaj
* Z `env.render()` metodo prikažemo trenutni frame
* Z `env.close()` metodo na varen način ugasnemo naše okolje
    
    > We are using the .env on the end of make to avoid training stopping at 200 iterations, which is the default for the new version of Gym
</div>

In [2]:
import gymnasium as gym
import time

env = gym.make("Taxi-v3", render_mode="human").env
env.reset(seed=123)
env.render()
time.sleep(3)
env.close()

<div class="not_clean">

Znotraj atributov `observation_space` in `action_space` lahko vidimo, da imamo 500 različnih **states** in 6 različnih akcij.
</div>

In [4]:
import gymnasium as gym
import time

env = gym.make("Taxi-v3", render_mode="human").env
env.reset(seed=123)
env.render()

print("State Space {}".format(env.observation_space))
print("Action Space {}".format(env.action_space))

time.sleep(3)
env.close()

State Space Discrete(500)
Action Space Discrete(6)


<div class="not_clean">

Vsak **state** ima unique številko, ki ga identificira. V našem primeru na sliki je to state **341**. Poleg tega lahko za vsak state vidimo kolikšen je reward za posamezno akcijo.
</div>

In [ ]:
import gymnasium as gym
import time

env = gym.make("Taxi-v3", render_mode="human")
env.reset(seed=123)

print(env.s)
print(env.P[env.s])
env.render()

time.sleep(3)
env.close()


<div class="not_clean">

```python
{0: [(1.0, 441, -1, False)], 
 1: [(1.0, 241, -1, False)], 
 2: [(1.0, 341, -1, False)], 
 3: [(1.0, 321, -1, False)], 
 4: [(1.0, 341, -10, False)], 
 5: [(1.0, 341, -10, False)]}
```
`{action: [(probability, nextstate, reward, done)]}`

* 0-5 predstavlja naše akcije
* probability - če se odločimo za to akcijo, kolikšna je možnost za premik v naslednji state. V Taxi primeru je to vedno 1
* nextstate - state v katerem bomo po opravljeni akciji
* reward - kolikšen reward prejmemo za našo akcijo
* done - ali bo ta akcija končala episodo, ali smo potnika odložili na pravi lokaciji
</div>

<div class="not_clean">

Če bi igrali mi, bi najkrajša pot bila sledeča:
</div>

In [ ]:
import gymnasium as gym
import time

env = gym.make("Taxi-v3", render_mode="human")
env.metadata["render_fps"] = 5
env.reset(seed=123)

total_reward = 0
steps = 0

actions = [1, 3, 3, 1, 1, 4, 0, 0, 2, 2, 2, 2, 1, 1, 5] # shortest path for seed 123

done = False
while not done:
    env.render()
    action = actions[steps]    
    observation, reward, done, truncated, info = env.step(action)
    
    total_reward += reward
    steps += 1

env.render()
time.sleep(2)
env.close()

print("Steps taken", steps)
print("Total reward", total_reward)


## Baseline - Random walking

<div class="not_clean">

Za začetek bomo ustvarili baseline model. Ustvarili bomo neskončno zanko, kjer konstantno gledamo kolikšen je kakšen reward za kakšno akcijo. Taxi se odloči za največjo nagrado. Če jih je več istih se bo taxi odločil za random akcijo.
</div>

In [ ]:
import gymnasium as gym
import random
import time

env = gym.make("Taxi-v3", render_mode="human")
env.metadata["render_fps"] = 200
env.reset(seed=123)

total_reward = 0
steps = 0

done = False
while not done:
    env.render()

    rewards = env.P[env.s]
    max_reward = max(rewards.values(), key=lambda x: x[0][2])[0][2] # find max reward in current state
    best_actions = {action:outcome for action, outcome in rewards.items() if outcome[0][2]==max_reward} # collect all actions that will give this max reward
    action = random.choice(list(best_actions.keys())) # select a random action out of these best actions
    
    observation, reward, done, truncated, info = env.step(action) # do the action
    
    total_reward += reward
    steps += 1

env.render()
time.sleep(2)
env.close()

print("Steps taken", steps)
print("Total reward", total_reward)


<div class="not_clean">

Končni rezultat zelo variira ampak imamo nek baseline katerega želimo izboljšati.
    
```
Total steps:  1222.2
Total reward:  -1201.2
```
</div>

## Q-Learning

<div class="not_clean">

Q-Learning omogoča agentu, da se preko nagrad in s časom nauči katere so akcije, ki skupaj vodijo do maximalne nagrade.
    
Za agenta bomo ustvarili **Q-Table** v kateri imamo kot vrstice vse možne **state** in kot stolpce vse možne **action**. Vsaka celica v tabeli drži **q-value**, ki nam pove kako kvalitetna je specifična akcija v specifičnem **state**. 
    
Na primer, ko Taxi pobere potnika tehnično ne dobi nobene nagrade. Vendar pa je ta akcija kvalitetna, saj Taxi ne more zmagati, če potnika ne pripelje na cilj. Q-value "poberi potnika" bi tako bila višja kot ostale akcije v tistem **state**.
</div>

<div class="not_clean">

Q-value se izračuna glede na enačbo:
</div>

$new \ Q(state, action) = (1 - \alpha) * Q(state, action) + \alpha(reward + \gamma maxQ(next \ state, all \ actions))$

* $Q(state, action)$ - q-value za state v katerem se trenutno nahajamo in akcijo katero bomo naredili
* $\alpha$ - predstavlja learning rate ($0 < \alpha \le 1$). Predstavlja za koliko želimo posodobiti novi q-value
* $reward$ - reward katerega smo prejeli, ko smo opravili naš action
* $\gamma$ - predstavlja **discount factor**. Ta nam pove koliko pomembnosti dajemo na prihodnjo nagrado oziroma Q vrednost. Vrednost blizu 1 pomeni, da želimo povdariti končni rezultat. Vrednosti proti 0 pomeni, da želimo gledati le trenutno nagrado (greedy policy).
* $maxQ(next \ state, all \ actions)$ - pogledamo v katerem stanju se bomo znašli po naši akciji. Nato vzememo največjo q-value za tiste `state-action` pare.

<div class="not_clean">

Q-value posodobimo glede na to kolikšna je trenutna vrednost in kolikšen je max q-value stanja v katerem se bomo znašli.
    
S takim posodabljanjem bo Taxi počasi našel optimalno pot katero mora izvesti.
</div>

-----

<div class="not_clean">

Na tak način bi se Taxi lahko overfittal in vedno izbiral samo eno in isto pot. Zato bomo dodali še faktor $\epsilon$ s katerim se bomo občasno odločili za "ne-optimalno" akcijo in tako raziskovali naš enviornment.
</div>

<div class="to_do">

Dodat primer brez epsiolona pa pol dodamo epsilon naknadno ko vidmo da ne izbere optimalne poti?
Al je pa to preveč povdarka na qlearningu.
</div>

### Koda

<div class="not_clean">

Q-table bomo inicializirali s samimi 0. Nato bomo začeli naš igro. S pomočjo `random.uniform(0,1)` bomo dobili random številko med 0 in 1. Če je številka manjša ali enaka epsilon bomo naredili random potezo.
    
Nato sledi posodobitev naše q-tabele.
</div>

In [ ]:
import random
import gymnasium as gym
import numpy as np
import time

env = gym.make("Taxi-v3", render_mode="human")
env.metadata["render_fps"] = 200

# Q-Table
q_table = np.zeros([env.observation_space.n, env.action_space.n])

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# Performance tracking
total_reward = 0
steps = 0

# Env setup
done = False
state, info = env.reset(seed=123)
while not done:
    env.render()

    if random.uniform(0,1) <= epsilon:
        action = env.action_space.sample() # Explore by making random action
    else:
        action = np.argmax(q_table[state]) # Be greedy and make best action
    next_state, reward, done, truncated, info = env.step(action)
    print("State", state, "Action", action, "Reward", reward, "Next state", next_state, "Done", done, "Turncated", truncated, "Step", steps) if steps % 100 == 0 else None

    # Update Q-Table
    old_value = q_table[state, action]
    next_max = np.max(q_table[next_state])
    new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
    q_table[state, action] = new_value


    # Update performance tracking
    total_reward += reward
    steps += 1

env.render()
time.sleep(2)
env.close()

print("Steps taken", steps)
print("Total reward", total_reward)
print(q_table[341])

<div class="not_clean">

Naš taxi se vrti v krogih in se ne uči zares. Dodajmo mu še veliko negativno nagrado, če igre ne zaključi v max 200 korakih.
</div>

In [ ]:
import random
import gymnasium as gym
import numpy as np
import time

env = gym.make("Taxi-v3", render_mode="human")
env.metadata["render_fps"] = 200

# Q-Table
q_table = np.zeros([env.observation_space.n, env.action_space.n])

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# Performance tracking
total_reward = 0
steps = 0

# Env setup
done = False
state, info = env.reset(seed=123)
while not done:
    env.render()

    if random.uniform(0,1) <= epsilon:
        action = env.action_space.sample() # Explore by making random action
    else:
        action = np.argmax(q_table[state]) # Be greedy and make best action
    next_state, reward, done, truncated, info = env.step(action)
    print("State", state, "Action", action, "Reward", reward, "Next state", next_state, "Done", done, "Turncated", truncated, "Step", steps) if steps % 20 == 0 else None
    # vvvv    HERE HERE HERE    vvvvv
    if truncated:
        done = True
        reward = -20
    # ^^^^    HERE HERE HERE    ^^^^

    # Update Q-Table
    old_value = q_table[state, action]
    next_max = np.max(q_table[next_state])
    new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
    q_table[state, action] = new_value


    # Update performance tracking
    total_reward += reward
    steps += 1

env.render()
time.sleep(2)
env.close()

print("Steps taken", steps)
print("Total reward", total_reward)
print(q_table[341])


<div class="not_clean">

Za naš primer smo na koncu izpisali q-values za našo začetno pozicijo - se pravi, katera akcija se nam najbolj izplača čisto na začetku.
    
`[-0.9948844  -0.9953616  -0.99496515 -2.8953616  -3.439      -4.0951    ]`
    
Glede na končno q-tabelo vidimo da so najboljše 3 akcije `down`, `up`, `right`. Vidimo, da se ni zares nekaj naučil. Želeli bi le eno zares dobro akcijo.
    
Smo pa taxi učili samo z eno episodo. Poizkusimo ga učiti tekom več igranj igre.
</div>

In [ ]:
import random
import gymnasium as gym
import numpy as np
# vvvv    HERE HERE HERE    vvvv
import matplotlib.pyplot as plt

env = gym.make("Taxi-v3", render_mode=None)
# ^^^^    HERE HERE HERE    ^^^^
env.metadata["render_fps"] = 300

# Q-Table
q_table = np.zeros([env.observation_space.n, env.action_space.n])

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# vvvv    HERE HERE HERE    vvvv
episodes = 10_001
total_rewards = []
total_steps = []
for ep in range(episodes):
    # ^^^^    HERE HERE HERE    ^^^^
    # Performance tracking
    total_reward = 0
    steps = 0

    # Env setup
    done = False
    state, info = env.reset()  # (seed=123)
    while not done:
        # vvvv    HERE HERE HERE    vvvv
        # env.render() # we dont need to render anything
        # ^^^^    HERE HERE HERE    ^^^^
        if random.uniform(0, 1) <= epsilon:
            action = env.action_space.sample()  # Explore by making random action
        else:
            action = np.argmax(q_table[state])  # Be greedy and make best action
        next_state, reward, done, truncated, info = env.step(action)
        if truncated:
            done = True
            reward = -20

        # Update Q-Table
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value
        state = next_state

        # Update performance tracking
        total_reward += reward
        steps += 1
    # vvvv    HERE HERE HERE    vvvv
    total_rewards.append(total_reward)
    total_steps.append(steps)

    if ep % 200 == 0:
        print("Episode", ep)
        print("Steps taken", steps)
        print("Total reward", total_reward)
        print()

env.close()
print(q_table[341])
np.save("models/taxi.npy", q_table)


fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)

ax1.plot(total_rewards[::50], c="lightskyblue")
ax1.set_ylabel("Rewards")

ax2.plot(total_steps[::50], c="pink")
ax2.set_xlabel("Episode")
ax2.set_ylabel("Steps")

fig.tight_layout()
plt.show()
# ^^^^    HERE HERE HERE    ^^^^

<div class="not_clean">

Po daljšem učenju dobimo sledeč rezultat glede najboljše začetne akcije:
    
`[ -2.6029108   -3.49428052  -2.66963255  -2.48236823 -11.49844851   -11.64223247]`
    
Tukaj vidimo, da je optimalna akcija `left`, kar je ena izmed optimalnih rešitev. 
    
Na naših grafih vidimo, da se je celotna nagrada izboljševala in, da je čas igranja upadal (kar pomeni, da se taxi ni vrtel v krogih ampak je dostavil potnika).
    
Če si pogledamo našo episodo sedaj:
</div>

In [ ]:
import gymnasium as gym
import numpy as np

env = gym.make("Taxi-v3", render_mode="human")
env.metadata["render_fps"] = 5

# Q-Table
q_table = np.load("models/taxi_good.npy")

episodes = 1

for ep in range(episodes):
    # Performance tracking
    total_reward = 0
    steps = 0

    # Env setup
    done = False
    state, info = env.reset(seed=123)
    while not done:
        action = np.argmax(q_table[state])  # Be greedy and make best action

        state, reward, done, truncated, info = env.step(action)
        if truncated:
            done = True
            reward = -20

        # Update performance tracking
        total_reward += reward
        steps += 1

    print("Episode", ep)
    print("Steps taken", steps)
    print("Total reward", total_reward)
    print()

env.render()
env.close()

<div class="not_clean">

Bolj optimalno kot to nebi mogl met.
   
</div>

<div class="not_clean">    
V našem primeru si lahko še pogledamo heat-map naše Q-Tabele. Višji Q-Value kot imamo, bolj bela je naša barva. 
</div>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

q_table = np.load("models/taxi_good.npy")

# Create the figure and axes
fig, ax = plt.subplots()
im = ax.imshow(q_table, cmap="hot", interpolation="none", aspect="auto")
plt.colorbar(im)
plt.title("Q-Table")

# Add column names to the heatmap
column_names = ["Down", "Up", "Right", "Left", "Pickup", "Dropoff"]
ax.set_xticks(np.arange(len(column_names)))
ax.set_xticklabels(column_names, rotation=45)

# Display the heatmap
plt.show()

<div class="not_clean">    

Zadeva je rahlo nepregledna zato vzemimo le vsa stanja v katerih se taxi znajde in vsa sosednja stanja.
    
Prvo pridobimo vsa stanja katera taxi obišče in vsa sosednja stanja.
</div>

In [ ]:
import gymnasium as gym
import numpy as np

env = gym.make("Taxi-v3", render_mode="human")
env.metadata["render_fps"] = 5

# Q-Table
q_table = np.load("models/taxi_good.npy")

# vvvv    HERE HERE HERE    vvvv
# Holds all states taxi visites and all neighboring states
states = set()
# ^^^^    HERE HERE HERE    ^^^^

# Env setup
done = False
state, info = env.reset(seed=123)
while not done:
    # vvvv    HERE HERE HERE    vvvv
    states.update([state])  # Add the state taxi is in
    neigbour_states = [
        v[0][1] for v in env.P[state].values()
    ]  # grab neighboring states
    states.update(neigbour_states)
    # ^^^^    HERE HERE HERE    ^^^^

    action = np.argmax(q_table[state])  # Be greedy and make best action

    state, reward, done, truncated, info = env.step(action)
    if truncated:
        done = True
        reward = -20

env.render()
env.close()

# vvvv    HERE HERE HERE    vvvv
print("States:", list(states))
# ^^^^    HERE HERE HERE    ^^^^


<div class="not_clean">    

States: `[1, 257, 137, 397, 17, 21, 277, 157, 421, 37, 297, 177, 441, 321, 197, 201, 77, 337, 341, 85, 217, 221, 97, 101, 357, 377, 237, 241, 117, 121]`
    
Sedaj pa si poonvon poglejmo heatmap ter kako se taxi premika po tej heat mapi.
</div>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import gymnasium as gym

states = [1, 257, 137, 397, 17, 21, 277, 157, 421, 37, 297, 177, 441, 321, 197, 201, 77, 337, 341, 85, 217, 221, 97, 101, 357, 377, 237, 241, 117, 121]


env = gym.make("Taxi-v3", render_mode="human")
env.metadata["render_fps"] = 5

# Q-Table
q_table = np.load("models/taxi_good.npy")

# Create the figure and axes
fig, ax = plt.subplots()
im = ax.imshow(q_table, cmap="hot", interpolation="none", aspect="auto")
plt.colorbar(im)
plt.title("Q-Table")
# Add column names to the heatmap
column_names = ["Down", "Up", "Right", "Left", "Pickup", "Dropoff"]
ax.set_xticks(np.arange(len(column_names)))
ax.set_xticklabels(column_names, rotation=45)
# Display the initial heatmap
plt.show(block=False)

# Env setup
done = False
state, info = env.reset(seed=123)
while not done:
    action = np.argmax(q_table[state])  # Be greedy and make best action

    matrix_ = q_table.copy()
    matrix_[state, action] = np.max(q_table)
    matrix_ = matrix_[states, :]  # select only our path states
    im.set_data(matrix_)
    plt.pause(0.5)

    state, reward, done, truncated, info = env.step(action)
    if truncated:
        done = True
        reward = -20

env.render()
env.close()

plt.close(fig)


<div class="not_clean">    
Pogledamo si še lahko kako se heatmap spreminja tekom učenja.
</div>

In [ ]:
import random
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt

env = gym.make("Taxi-v3", render_mode=None)
env.metadata["render_fps"] = 300

# Q-Table
q_table = np.zeros([env.observation_space.n, env.action_space.n])

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1


states = [
    1,
    257,
    137,
    397,
    17,
    21,
    277,
    157,
    421,
    37,
    297,
    177,
    441,
    321,
    197,
    201,
    77,
    337,
    341,
    85,
    217,
    221,
    97,
    101,
    357,
    377,
    237,
    241,
    117,
    121,
]

# Create the figure and axes
fig, ax = plt.subplots()
im = ax.imshow(q_table, cmap="hot", interpolation="none", aspect="auto")
plt.colorbar(im)
plt.title("Q-Table")
# Add column names to the heatmap
column_names = ["Down", "Up", "Right", "Left", "Pickup", "Dropoff"]
ax.set_xticks(np.arange(len(column_names)))
ax.set_xticklabels(column_names, rotation=45)
# Display the initial heatmap
plt.show(block=False)

episodes = 10_001
total_rewards = []
total_steps = []
for ep in range(episodes):
    # Performance tracking
    total_reward = 0
    steps = 0

    # Env setup
    done = False
    state, info = env.reset(seed=123)
    while not done:
        if random.uniform(0, 1) <= epsilon:
            action = env.action_space.sample()  # Explore by making random action
        else:
            action = np.argmax(q_table[state])  # Be greedy and make best action
        next_state, reward, done, truncated, info = env.step(action)
        if truncated:
            done = True
            reward = -20

        # Update Q-Table
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value
        state = next_state

        # Update performance tracking
        total_reward += reward
        steps += 1
        
    total_rewards.append(total_reward)
    total_steps.append(steps)

    if ep % 200 == 0:
        print("Episode", ep)
        print("Steps taken", steps)
        print("Total reward", total_reward)
        print()
        matrix_ = q_table.copy()
        matrix_ = matrix_[states, :]  # select only our path states
        im.set_data(matrix_)
        plt.pause(0.2)


env.close()
plt.close(fig)

<div class="not_clean">

Q-Learning je eden izmed najbolj preprostih Reinforcement Learning algoritmov. Problem se pojavi, ko število **states** postane izredno veliko saj se velikost Q-Table neznansko poveča.
    
Za reševanje tega problema se uporablja nevronske mreže in tako dobimo **Deep Q Learning**.
</div>

<div class="to_do">

Ta heatmap tekom učenja ni najlepše za vidt kaj se dogaja.. težko je razbrat kaj gledaš
</div>

-----

# Deep Q Learning

<div class="not_clean">

Pri Deep Q Learning-u sedaj uporabljamo nevronsko mrežo znotraj katere hranimo Q vrednosti. Vhodne vrednosti so naš state vrednosti. Izhodne vredonsti so naše Q vrednosti. Sedaj se lahko naučimo bolj kompleksna okolja. 
    
Deep Q Learning je sposoben tudi opraviti dobre akcije v stanjih v katerih se še ni znašel. Ve, da je že bil v podobni situaciji in tako lahko opravi bolj optimalno akcijo kot pa Q-Table.

</div>

## CartPole

<div class="not_clean">

Za naš primer bomo ponovno uporabili knjižnjico Gymnasium. Tokrat bomo vzeli igro `CartPole-v1`.
    
Cilj igre je balancirati palco na premikajoči škatli.
</div>

[CartPole-v1](https://gymnasium.farama.org/environments/classic_control/cart_pole/)

<div>
<img src="images/cartpole.png" width="500"/>
</div>

### Action Space

<div class="not_clean">

Na voljo imamo 2 akciji:
* 0 - premik škatle v levo
* 1 - premik škatle v desno
</div>

### Observation Space

<div class="not_clean">

Naš state je sestavljen iz 4 vrednosti:
* 0 - `cart position` - pozicija škatle, ki se giblje med -4.8 do 4.8
* 1 - `cart velocity` - hitrost škatle, ki se giblje med -inf in inf.
* 2 - `pole angle` - kot palice, ki se giblje med -0.418 rad in 0.418 rad
* 3 - `pole angular velocity` - kotna hitrost palice, ki se giblje med -inf in inf
    
Treba je paziti, saj ni mogoč doseči vseh stanj. Igra se zaključi, če se škatla premakne preko pozicij -2.4 oziroma +2.4. Prav tako se igra zaključi, če je kot palice večji kot -0.2095 oziroma 0.2095
</div>

### Rewards

<div class="not_clean">

Cil igre je čim dlje časa balancirati palico. Za vsak korak, ki ga napravimo, dobimo nagrado `+1`.
</div>

---

<div class="not_clean">

Poglejmo si, če model dela random akcije.
</div>

In [ ]:
import gymnasium as gym

env = gym.make("CartPole-v1", render_mode="human")
env.metadata["render_fps"] = 24
print("Action space:", env.action_space)
print("Observation space:", env.observation_space)

episodes = 5

for ep in range(episodes):
    # Performance tracking
    total_reward = 0

    done = False
    env.reset()
    while not done:
        next_state, reward, done, truncated, info = env.step(env.action_space.sample())
        if truncated:
            done = True

        # Performance tracking
        total_reward += reward
    
    print("Episode: ", ep)
    print("Total reward: ", total_reward)

env.close()

---

<div class="not_clean">

Ustvarimo sedaj DQN Agenta, ki bo opravljal random akcije - agent se še ne bo učil.
</div>

In [ ]:
import gymnasium as gym

import random
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

env = gym.make("CartPole-v1", render_mode="human")
env.metadata["render_fps"] = 24


class DQNAgent:
    def __init__(self, env):
        self.env = env
        # Hyperparameters
        self.epsilon = 0.1

        self.create_model()

    def create_model(self):
        self.model = Sequential()
        self.model.add(Dense(32, input_dim=self.env.observation_space.shape[0], activation="relu"))
        self.model.add(Dense(16, activation="relu"))
        self.model.add(Dense(self.env.action_space.n, activation="linear"))
        self.model.compile(loss="mse", optimizer=Adam(lr=0.001))

    def action(self, state):
        if np.random.random() <= self.epsilon:
            return env.action_space.sample()
        else:
            return np.argmax(self.model.predict(state)[0])

agent = DQNAgent(env)
print(agent.model.summary())

episodes = 5

for ep in range(episodes):
    # Performance tracking
    total_reward = 0

    done = False
    state, _ = env.reset()
    while not done:

        state = np.reshape(state, [1, env.observation_space.shape[0]])
        action = agent.action(state)
        state, reward, done, truncated, info = env.step(action)
        if truncated:
            done = True

        # Performance tracking
        total_reward += reward
    
    print("Episode: ", ep)
    print("Total reward: ", total_reward)

env.close()


---

<div class="not_clean">

Modela nočemo fit-ati na eno trenutno izkušnjo/nagrado saj tako lahko začne pozabljati, kaj vse se je že naučil. Zato bomo uporabili replay memory. To je kot nek približek biološkega spomina. V njega shranjujemo naše izkušnje iz katerih se nato lahko za nazaj učimo.

To pomeni, da bomo naše states in nagrade shranjevali v nek list omejene velikost. Vsak korak bomo vzeli random batch naših stanj in naredili fit čez ta batch.
</div>



In [ ]:
import random
import gymnasium as gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import matplotlib.pyplot as plt

env = gym.make("CartPole-v1", render_mode=None)
env.metadata["render_fps"] = 200


class DQNAgent:
    def __init__(self, env):
        self.env = env
        # Hyperparameters
        self.epsilon = 0.1
        # vvvv    HERE HERE HERE    vvvv
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95
        # ^^^^    HERE HERE HERE    ^^^^

        self.create_model()

    def create_model(self):
        self.model = Sequential()
        self.model.add(Dense(32, input_dim=self.env.observation_space.shape[0], activation="relu"))
        self.model.add(Dense(16, activation="relu"))
        self.model.add(Dense(self.env.action_space.n, activation="linear")) # Try something else than linear
        self.model.compile(loss="mse", optimizer=Adam(lr=0.001))

    def action(self, state):
        if np.random.random() <= self.epsilon:
            return env.action_space.sample()
        else:
            return np.argmax(self.model.predict(state)[0])
    
    # vvvv    HERE HERE HERE    vvvv
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self):
        batch_size = 64

        if len(self.memory) < batch_size:
            return None # We don't have enough data to train
        
        minibatch = random.sample(self.memory, batch_size)

        state = np.zeros((batch_size, self.env.observation_space.shape[0]))
        next_state = np.zeros((batch_size, self.env.observation_space.shape[0]))
        action, reward, done = [], [], []

        for i in range(batch_size):
            state[i] = minibatch[i][0]
            action.append(minibatch[i][1])
            reward.append(minibatch[i][2])
            next_state[i] = minibatch[i][3]
            done.append(minibatch[i][4])

        target = self.model.predict(state)
        target_next = self.model.predict(next_state)

        for i in range(batch_size):
            if done[i]:
                target[i][action[i]] = reward[i]
            else:
                target[i][action[i]] = reward[i] + self.gamma * np.amax(target_next[i])

        self.model.fit(state, target, batch_size=batch_size, verbose=0)
    # ^^^^    HERE HERE HERE    ^^^^



agent = DQNAgent(env)
print(agent.model.summary())

episodes = 25
# vvvv    HERE HERE HERE    vvvv
total_rewards = []
# ^^^^    HERE HERE HERE    ^^^^

for ep in range(episodes):
    # Performance tracking
    total_reward = 0

    done = False
    state, _ = env.reset()
    state = np.reshape(state, [1, env.observation_space.shape[0]])

    while not done:
        action = agent.action(state)
        
        next_state, reward, done, truncated, info = env.step(action)
        next_state = np.reshape(next_state, [1, env.observation_space.shape[0]])
        # vvvv    HERE HERE HERE    vvvv
        if done:
            # If agent lost give big negative reward, because default reward is 1
            reward = -10
        elif truncated:
            # Don't give negative reward, but stop playing so we don't have infinite episode
            done = True
       
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        agent.replay()
        # ^^^^    HERE HERE HERE    ^^^^

        # Performance tracking
        total_reward += reward
    
    total_rewards.append(total_reward)
    print("Episode: ", ep)
    print("Total reward: ", total_reward)


env.close()
# vvvv    HERE HERE HERE vvvv
agent.model.save("models/cartpole.h5")

# Plot total rewards
plt.plot(total_rewards)
plt.show()
# ^^^^    HERE HERE HERE    ^^^^

<div>
<img src="./images/cartpole_learning.png" width="500">
</div>

<div class="not_clean">

Test da pogledamo zdej par episod ko igra
</div>

In [ ]:
import gymnasium as gym
import numpy as np
from keras.models import load_model


env = gym.make("CartPole-v1", render_mode="human")
env.metadata["render_fps"] = 30

agent = load_model("models/cartpole_good.h5")

episodes = 5

for ep in range(episodes):
    # Performance tracking
    total_reward = 0

    done = False
    state, _ = env.reset()
    state = np.reshape(state, [1, env.observation_space.shape[0]])

    while not done:
        action = np.argmax(agent.predict(state)[0])        
        state, reward, done, truncated, info = env.step(action)
        state = np.reshape(state, [1, env.observation_space.shape[0]])
        
        if truncated:
            # Don't give negative reward, but stop playing so we don't have infinite episode
            done = True
        
        # Performance tracking
        total_reward += reward
    
    print("Episode: ", ep)
    print("Total reward: ", total_reward)


env.close()

---

# Stable baseline 3

<div class="not_clean">

Da ne potrebujemo sami pisati vseh alogiritmov obstajajo za to že narejene knjižnjice.

Ena izmed knjižnjic je [Stable baseline 3](https://stable-baselines.readthedocs.io/en/master/), ki znotraj sebe implementira različne algoritme in nudi podporo za Gym enviornments.
</div>

Installing:
* `pip install stable-baselines3`

<div class="not_clean">

Za začetek bomo ponovili naš CartPole primer.

Prvo uvozimo Gymnasium knjižnjico s katero bomo definirali okolje katerega bomo uporabili. Nato preko stable baselines knjiznjice uvozimo model katerega želimo uporabiti. V našem primeru bomo začeli z že znanim DQN. 

Nato ustvarimo okolje in ustvarimo agenta. Uporabili bomo `MlpPolicy` - navaden deep neural network.

Nato bomo model trenirali za 100_000 korakov.

Model bomo shranili in za konec bomo prikazali nekaj episodo igranja.
</div>

<div class="not_clean">

Kaj pomeni **deterministic** parameter?
    
> This parameter corresponds to **"Whether to use deterministic or stochastic actions"**. So the thing is when you are selecting an action according to given state, the **actor_network** gives you a probability distribution. For example for two possible actions **a1** and **a2**: `[0.25, 0.75]`. If you use `deterministic=True`, the result will be action **a2** since it has more probability. In the case of `deterministic=False`, the result action will be selected with given probabilities `[0.25, 0.75]`.
</div>




In [ ]:
import gymnasium as gym

from stable_baselines3 import DQN

model = DQN("MlpPolicy", "CartPole-v1").learn(100_000, progress_bar=True)
model.save("models/DQN_SB3")

env = gym.make("CartPole-v1", render_mode="human")
env.metadata["render_fps"] = 30

episodes = 5

for ep in range(episodes):
    # Performance tracking
    total_reward = 0

    done = False
    obs, _ = env.reset()
    while not done:
        action, _state = model.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = env.step(action)
        if truncated:
            done = True
        # Performance tracking
        total_reward += reward

    print("Episode: ", ep)
    print("Total reward: ", total_reward)


<div class="not_clean">

Da uporabimo shranjen model imamo preprosto load funkcijo, ki naloži `.zip` datoteko shranjenega modela.
</div>

In [ ]:
import gymnasium as gym

from stable_baselines3 import DQN

env = gym.make("CartPole-v1", render_mode="human")
env.metadata["render_fps"] = 30

model = DQN.load("models/DQN_SB3_Best")


episodes = 5

for ep in range(episodes):
    # Performance tracking
    total_reward = 0

    done = False
    obs, _ = env.reset()
    while not done:
        action, _state = model.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = env.step(action)
        if truncated:
            done = True
        # Performance tracking
        total_reward += reward

    print("Episode: ", ep)
    print("Total reward: ", total_reward)


---

# Drugi algoritmi

<div class="not_clean">

Poleg DQN imamo znotraj knjižnjice implementirane še druge algoritme.

V splošnem jih je težko razdeliti v posamezne kategorije ampak v grobem se delijo na sledeč način:

* **Model-Based**
* **Model-Free**
</div>

![rl algos](./images/rl_algos_classification.svg)

## Model-based vs. Model-free

<div class="not_clean">

Glavno vprašanje je **ali ima agent dostop do modela okolja, oziroma se model okolja nauči**? Model okolja v tem kontekstu pomenu funkcijo, ki definira prehode med stanji in nagrade.
</div>


<div class="not_clean">

Glavna prednost, če imamo model okolja je, da ta omogoča agentu planiranje v naprej. Agent lahko vidi katere možnosti mu bodo na voljo v prihodnosti in se odloča na podlagi tega planiranja. Znan primer **model-based** algoritma je AlphaZero.

Glavna ovira pri teh algoritmih je, da agent ponavadi nima dostopa do **gronud-truth modela okolja**. Kar pomeni, da če se agent želi naučiti model okolja, bo to moral doseči preko svojih izkušenj. Eden izmed problem tukaj je potencialno zelo dolg čas učenja. Drugi problem je, da se agent ne nauči pravilnega modela okolja ampak nek njegov približek, ki vsebuje določen *bias / pristranskost*. Ta bias agent izkoristi, kar privede do zelo dobri rezultatov tekom učenja, vendar pa se agent nato v realnom okolju izkaže zelo slabo.
</div>


<div class="not_clean">

**Model-free** alogiritmi omogočajo lažjo implementacijo in lažje spreminjanje / *tuning*. Zato se večinoma uporablja te algoritme.
</div>

## Policy learning vs. Value learning

<div class="not_clean">

Druga točka v kateri se razlikujejo algoritmi je **kaj naj se naučijo?**

V primeru **model-free** algoritmov imamo v glavnem dve možnosti:
* **Policy optimization** - te algoritmi direktno optimizirajo **policy funkcijo**, ki mapira stanja v akcije, ki bodo imele najvišjo končno nagrado.

* **Q-learning** - te algoritmi se učijo **action-value funkcijo**, ki pove kako dobra je določena akcija
</div>

<div class="not_clean">

**Policy optimization** algoritmi so bolj stabilni in zanesljivi. Na drugi strani pa **Q-learning** algoritmi veliko bolje izkoriščajo podatke katere imajo na voljo.
</div>


---

<div class="not_clean">

Znotraj naše knjižnjice lahko tako najdemo algoritme kot so:
* DQN - katerega smo že uporabili
* A2C - kjer naenkrat treniramo dve mreži. Prva predstavlja **actor** in je naš agenti, ki dela akcije. Druga mreža je **critic**, ki agentu podaja feedback glede njegovih akcij
* PPO - kjer algoritem posodablja svoj policy v zelo majhnih korakih
</div>

In [ ]:
import gymnasium as gym

from stable_baselines3 import PPO

model = PPO("MlpPolicy", "CartPole-v1").learn(10_000, progress_bar=True)
model.save("models/ppo")

env = gym.make("CartPole-v1", render_mode="human")
env.metadata["render_fps"] = 30

episodes = 5

for ep in range(episodes):
    # Performance tracking
    total_reward = 0

    done = False
    obs, _ = env.reset()
    while not done:
        action, _state = model.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = env.step(action)
        if truncated:
            done = True
        # Performance tracking
        total_reward += reward

    print("Episode: ", ep)
    print("Total reward: ", total_reward)


<div class="not_clean">

Poglejmo si primerjavo med DQN, A2C in PPO, kjer smo vsakega trenirali 10 000 korakov.
</div>

In [ ]:
import gymnasium as gym

from stable_baselines3 import DQN, A2C, PPO

#model = DQN("MlpPolicy", "CartPole-v1").learn(10_000, progress_bar=True)
model = A2C("MlpPolicy", "CartPole-v1").learn(10_000, progress_bar=True)
#model = PPO("MlpPolicy", "CartPole-v1").learn(10_000, progress_bar=True)

env = gym.make("CartPole-v1", render_mode="human")
env.metadata["render_fps"] = 30

episodes = 5
total_rewards = []

for ep in range(episodes):
    # Performance tracking
    total_reward = 0

    done = False
    obs, _ = env.reset()
    while not done:
        action, _state = model.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = env.step(action)
        if truncated:
            done = True
        # Performance tracking
        total_reward += reward

    total_rewards.append(total_reward)
    print("Episode: ", ep)
    print("Total reward: ", total_reward)

print("Average reward: ", sum(total_rewards) / len(total_rewards))

<div class="not_clean">

Vidimo, da sta A2C in PPO dosti boljša od DQN.
</div>

---

<div class="not_clean">

Razlike med algoritmi so tudi v tem v kakšnih okoljih jih uporabljamo.
</div>

<table class="docutils align-default">
<thead>
<tr class="row-odd"><th class="head"><p>Name</p></th>
<th class="head"><p><code class="docutils literal notranslate"><span class="pre">Box</span></code></p></th>
<th class="head"><p><code class="docutils literal notranslate"><span class="pre">Discrete</span></code></p></th>
<th class="head"><p><code class="docutils literal notranslate"><span class="pre">MultiDiscrete</span></code></p></th>
<th class="head"><p><code class="docutils literal notranslate"><span class="pre">MultiBinary</span></code></p></th>
<th class="head"><p>Multi Processing</p></th>
</tr>
</thead>
<tbody>
<tr class="row-even"><td><p>ARS <a class="footnote-reference brackets" href="#f1" id="id1" role="doc-noteref"><span class="fn-bracket">[</span>1<span class="fn-bracket">]</span></a></p></td>
<td><p>✔️</p></td>
<td><p>✔️</p></td>
<td><p>❌</p></td>
<td><p>❌</p></td>
<td><p>✔️</p></td>
</tr>
<tr class="row-odd"><td><p>A2C</p></td>
<td><p>✔️</p></td>
<td><p>✔️</p></td>
<td><p>✔️</p></td>
<td><p>✔️</p></td>
<td><p>✔️</p></td>
</tr>
<tr class="row-even"><td><p>DDPG</p></td>
<td><p>✔️</p></td>
<td><p>❌</p></td>
<td><p>❌</p></td>
<td><p>❌</p></td>
<td><p>✔️</p></td>
</tr>
<tr class="row-odd"><td><p>DQN</p></td>
<td><p>❌</p></td>
<td><p>✔️</p></td>
<td><p>❌</p></td>
<td><p>❌</p></td>
<td><p>✔️</p></td>
</tr>
<tr class="row-even"><td><p>HER</p></td>
<td><p>✔️</p></td>
<td><p>✔️</p></td>
<td><p>❌</p></td>
<td><p>❌</p></td>
<td><p>✔️</p></td>
</tr>
<tr class="row-odd"><td><p>PPO</p></td>
<td><p>✔️</p></td>
<td><p>✔️</p></td>
<td><p>✔️</p></td>
<td><p>✔️</p></td>
<td><p>✔️</p></td>
</tr>
<tr class="row-even"><td><p>QR-DQN <a class="footnote-reference brackets" href="#f1" id="id2" role="doc-noteref"><span class="fn-bracket">[</span>1<span class="fn-bracket">]</span></a></p></td>
<td><p>❌</p></td>
<td><p>️ ✔️</p></td>
<td><p>❌</p></td>
<td><p>❌</p></td>
<td><p>✔️</p></td>
</tr>
<tr class="row-odd"><td><p>RecurrentPPO <a class="footnote-reference brackets" href="#f1" id="id3" role="doc-noteref"><span class="fn-bracket">[</span>1<span class="fn-bracket">]</span></a></p></td>
<td><p>✔️</p></td>
<td><p>✔️</p></td>
<td><p>✔️</p></td>
<td><p>✔️</p></td>
<td><p>✔️</p></td>
</tr>
<tr class="row-even"><td><p>SAC</p></td>
<td><p>✔️</p></td>
<td><p>❌</p></td>
<td><p>❌</p></td>
<td><p>❌</p></td>
<td><p>✔️</p></td>
</tr>
<tr class="row-odd"><td><p>TD3</p></td>
<td><p>✔️</p></td>
<td><p>❌</p></td>
<td><p>❌</p></td>
<td><p>❌</p></td>
<td><p>✔️</p></td>
</tr>
<tr class="row-even"><td><p>TQC <a class="footnote-reference brackets" href="#f1" id="id4" role="doc-noteref"><span class="fn-bracket">[</span>1<span class="fn-bracket">]</span></a></p></td>
<td><p>✔️</p></td>
<td><p>❌</p></td>
<td><p>❌</p></td>
<td><p>❌</p></td>
<td><p>✔️</p></td>
</tr>
<tr class="row-odd"><td><p>TRPO  <a class="footnote-reference brackets" href="#f1" id="id5" role="doc-noteref"><span class="fn-bracket">[</span>1<span class="fn-bracket">]</span></a></p></td>
<td><p>✔️</p></td>
<td><p>✔️</p></td>
<td><p>✔️</p></td>
<td><p>✔️</p></td>
<td><p>✔️</p></td>
</tr>
<tr class="row-even"><td><p>Maskable PPO <a class="footnote-reference brackets" href="#f1" id="id6" role="doc-noteref"><span class="fn-bracket">[</span>1<span class="fn-bracket">]</span></a></p></td>
<td><p>❌</p></td>
<td><p>✔️</p></td>
<td><p>✔️</p></td>
<td><p>✔️</p></td>
<td><p>✔️</p></td>
</tr>
</tbody>
</table>

<div class="not_clean">

* `Box` predstavlja vse možne akcije v določenemu action space (**continous**). Za primer lahko vzememo vožnjo avtomobila, kjer lahko na gas pritisnemo malo, močneje, zelo močno, itd.
* `Discrete` predstavlja seznam možnih akcij, kjer lahko v določenm trenutku opravimo eno izmed akcij. To smo imeli v primeru našega taxija. V določenem trenutku som lahko naredili eno specifično akcijo.
* `MultiDiscrete` - določena okolja imajo več seznamov diskretnih akcij. Za primer lahko vzememo upravljanje telesa. V določenem trenutku lahko premaknemo roko in/ali nogo.
* `MultiBinary` - podobno kot `MultiDiscrete`, le da imamo za vsako akcijo 2 možnosti.
</div>

---

# Custom Environments

<div class="not_clean">

Znotraj knjiznjice Stable Baselines lahko tudi ustvarimo naš custom environment.
</div>

<div class="not_clean">

Za naš primer bomo ustvarili igro **snake** (kača).
    
Kodo lahko najdemo na internetu. Za začetek samo prekopirajmo okolje in ga sami odigrajmo.

Tipke so **W, S, A, D**.
</div>

In [ ]:
# source: https://github.com/TheAILearner/Snake-Game-using-OpenCV-Python/blob/master/snake_game_using_opencv.ipynb
import numpy as np
import cv2
import random
import time


def collision_with_apple(apple_position, score):
    apple_position = [random.randrange(1, 50) * 10, random.randrange(1, 50) * 10]
    score += 1
    return apple_position, score


def collision_with_boundaries(snake_head):
    if (
        snake_head[0] >= 500
        or snake_head[0] < 0
        or snake_head[1] >= 500
        or snake_head[1] < 0
    ):
        return 1
    else:
        return 0


def collision_with_self(snake_position):
    snake_head = snake_position[0]
    if snake_head in snake_position[1:]:
        return 1
    else:
        return 0


img = np.zeros((500, 500, 3), dtype="uint8")
# Initial Snake and Apple position
snake_position = [[250, 250], [240, 250], [230, 250]]
apple_position = [random.randrange(1, 50) * 10, random.randrange(1, 50) * 10]

score = 0
prev_button_direction = 1
button_direction = 1

snake_head = [250, 250]
while True:
    cv2.imshow("a", img)
    cv2.waitKey(1)
    img = np.zeros((500, 500, 3), dtype="uint8")
    # Display Apple
    cv2.rectangle(
        img,
        (apple_position[0], apple_position[1]),
        (apple_position[0] + 10, apple_position[1] + 10),
        (0, 0, 255),
        3,
    )
    # Display Snake
    for position in snake_position:
        cv2.rectangle(
            img,
            (position[0], position[1]),
            (position[0] + 10, position[1] + 10),
            (0, 255, 0),
            3,
        )

    # Takes step after fixed time
    t_end = time.time() + 0.05
    k = -1
    while time.time() < t_end:
        if k == -1:
            k = cv2.waitKey(1)
        else:
            continue

    # 0-Left, 1-Right, 3-Up, 2-Down, q-Break
    # a-Left, d-Right, w-Up, s-Down

    if k == ord("a") and prev_button_direction != 1:
        button_direction = 0
    elif k == ord("d") and prev_button_direction != 0:
        button_direction = 1
    elif k == ord("w") and prev_button_direction != 2:
        button_direction = 3
    elif k == ord("s") and prev_button_direction != 3:
        button_direction = 2
    elif k == ord("q"):
        break
    else:
        button_direction = button_direction
    prev_button_direction = button_direction

    # Change the head position based on the button direction
    if button_direction == 1:
        snake_head[0] += 10
    elif button_direction == 0:
        snake_head[0] -= 10
    elif button_direction == 2:
        snake_head[1] += 10
    elif button_direction == 3:
        snake_head[1] -= 10

    # Increase Snake length on eating apple
    if snake_head == apple_position:
        apple_position, score = collision_with_apple(apple_position, score)
        snake_position.insert(0, list(snake_head))

    else:
        snake_position.insert(0, list(snake_head))
        snake_position.pop()

    # On collision kill the snake and print the score
    if (
        collision_with_boundaries(snake_head) == 1
        or collision_with_self(snake_position) == 1
    ):
        font = cv2.FONT_HERSHEY_SIMPLEX
        img = np.zeros((500, 500, 3), dtype="uint8")
        cv2.putText(
            img,
            "Your Score is {}".format(score),
            (140, 250),
            font,
            1,
            (255, 255, 255),
            2,
            cv2.LINE_AA,
        )
        cv2.imshow("a", img)
        cv2.waitKey(0)
        break

cv2.destroyAllWindows()

<div class="not_clean">

Našo igro moramo prvo pretvoriti v Gymnasium enviornment.

To pomeni, da moramo dedovati od `gym.Env` class-a in implementirati določene metode.
</div>


```python
import gymnasium as gym
import numpy as np
from gymnasium import spaces


class CustomEnv(gym.Env):
    """Custom Environment that follows gym interface."""

    metadata = {"render_modes": ["human"], "render_fps": 30}

    def __init__(self, arg1, arg2, ...):
        super().__init__()
        # Define action and observation space
        # They must be gym.spaces objects
        # Example when using discrete actions:
        self.action_space = spaces.Discrete(N_DISCRETE_ACTIONS)
        # Example for using image as input (channel-first; channel-last also works):
        self.observation_space = spaces.Box(low=0, high=255,
                                            shape=(N_CHANNELS, HEIGHT, WIDTH), dtype=np.uint8)

    def step(self, action):
        ...
        return observation, reward, terminated, truncated, info

    def reset(self, seed=None, options=None):
        ...
        return observation, info

    def render(self):
        ...

    def close(self):
        ...
```

---

<div class="not_clean">

Za začetek bomo definirali naš action space in observation space.

Napravimo lahko 4 diskretne akcije (ali up, ali down, ali left ali right).

Kot observation space je mamljivo preprosto vzeti naši sliko. Vendar to pomeni, da se more agent dodatno naučiti kaj slika predstavlja (kaj na sliki predstavlja kačo, kaj predstavlja jabolko, itd.). Boljše je, če lahko sami definiramo določene metrike.

V našem primeru bomo v enem stanju imeli podatke za:
* is apple to the left of head
* is apple to the right of head
* is apple above head
* is apple below head
* is wall or tail to the left of head
* is wall or tail to the right of head
* is wall or tail above head
* is wall or tail below head
</div>

In [ ]:
import random

import gymnasium as gym
import numpy as np
from gymnasium import spaces


def collision_with_apple(apple_position, score):
    apple_position = [random.randrange(1, 50) * 10, random.randrange(1, 50) * 10]
    score += 1
    return apple_position, score


def collision_with_boundaries(snake_head):
    if (
        snake_head[0] >= 500
        or snake_head[0] < 0
        or snake_head[1] >= 500
        or snake_head[1] < 0
    ):
        return 1
    else:
        return 0


def collision_with_self(snake_position):
    snake_head = snake_position[0]
    if snake_head in snake_position[1:]:
        return 1
    else:
        return 0


class SnekEnv(gym.Env):
    metadata = {"render_modes": ["human"], "render_fps": 30}

    def __init__(self):
        super(SnekEnv, self).__init__()
        # Define action and observation space
        # They must be gym.spaces objects
        # Example when using discrete actions:
        self.action_space = spaces.Discrete(4)
        # Example for using image as input (channel-first; channel-last also works):
        self.observation_space = spaces.Box(low=0, high=1, shape=(8,), dtype=np.float32)

    def step(self, action):
        pass

    def reset(self, seed=None, options=None):
        pass

    def render(self, mode="human"):
        pass

    def close(self):
        pass

<div class="not_clean">

Naslednjo stvar katero bomo dodali je `reset()` metoda. Ta preprosto ponastavi naše okolje.
    
Dodatno bomo že sedaj pripravili funkcijo, ki nam zgradi naš observation (vrne state v katerem se nahajamo).
</div>

In [ ]:
import random

import gymnasium as gym
import numpy as np
from gymnasium import spaces


def collision_with_apple(apple_position, score):
    apple_position = [random.randrange(1, 50) * 10, random.randrange(1, 50) * 10]
    score += 1
    return apple_position, score


def collision_with_boundaries(snake_head):
    if (
        snake_head[0] >= 500
        or snake_head[0] < 0
        or snake_head[1] >= 500
        or snake_head[1] < 0
    ):
        return 1
    else:
        return 0


def collision_with_self(snake_position):
    snake_head = snake_position[0]
    if snake_head in snake_position[1:]:
        return 1
    else:
        return 0


class SnekEnv(gym.Env):
    metadata = {"render_modes": ["human"], "render_fps": 30}

    def __init__(self):
        super(SnekEnv, self).__init__()
        # Define action and observation space
        # They must be gym.spaces objects
        # Example when using discrete actions:
        self.action_space = spaces.Discrete(4)
        # Example for using image as input (channel-first; channel-last also works):
        self.observation_space = spaces.Box(low=0, high=1, shape=(8,), dtype=np.float32)

    def step(self, action):
        pass

    def reset(self, seed=None, options=None):
        # vvvv    HERE HERE HERE    vvvv
        self.img = np.zeros((500, 500, 3), dtype="uint8")

        # Initial Snake and Apple position
        self.snake_position = [[250, 250], [240, 250], [230, 250]]
        self.apple_position = [
            random.randrange(1, 50) * 10,
            random.randrange(1, 50) * 10,
        ]

        self.score = 0
        self.snake_head = self.snake_position[0].copy()

        self.done = False
        self.truncated = False

        observation = self.constructObservation()

        return observation, {}  # observation, info
        # ^^^^    HERE HERE HERE    ^^^^

    def render(self, mode="human"):
        pass

    def close(self):
        pass

    # vvvv    HERE HERE HERE    vvvv
    def constructObservation(self):
        # Used to create a current observation
        """Observation holds: (all 1 or 0 values)
        * is apple to the left of head
        * is apple to the right of head
        * is apple above head
        * is apple below head
        * is wall or tail to the left of head
        * is wall or tail to the right of head
        * is wall or tail above head
        * is wall or tail below head
        """
        observation = []

        if self.apple_position[0] < self.snake_head[0]:  # to the left
            observation.extend([1, 0])
        else:
            observation.extend([0, 1])

        if self.apple_position[1] < self.snake_head[1]:  # above
            observation.extend([1, 0])
        else:
            observation.extend([0, 1])

        positions = self.snake_position.copy()
        head = positions[0].copy()

        # Check left
        head_left = [head[0] - 10, head[1]]
        positions[0] = head_left
        o = collision_with_boundaries(head_left) or collision_with_self(positions)
        observation.append(o)

        # Check right
        head_right = [head[0] + 10, head[1]]
        positions[0] = head_right
        o = collision_with_boundaries(head_right) or collision_with_self(positions)
        observation.append(o)

        # Check above
        head_above = [head[0], head[1] - 10]
        positions[0] = head_above
        o = collision_with_boundaries(head_above) or collision_with_self(positions)
        observation.append(o)

        # Check below
        head_below = [head[0], head[1] + 10]
        positions[0] = head_below
        o = collision_with_boundaries(head_below) or collision_with_self(positions)
        observation.append(o)

        observation = np.array(observation, dtype=np.float32)
        return observation
        # ^^^^    HERE HERE HERE    ^^^^


<div class="not_clean">

Nato bomo dodali `render()` metodo.
</div>

In [ ]:
import random

import gymnasium as gym
import numpy as np
from gymnasium import spaces
import cv2


def collision_with_apple(apple_position, score):
    apple_position = [random.randrange(1, 50) * 10, random.randrange(1, 50) * 10]
    score += 1
    return apple_position, score


def collision_with_boundaries(snake_head):
    if (
        snake_head[0] >= 500
        or snake_head[0] < 0
        or snake_head[1] >= 500
        or snake_head[1] < 0
    ):
        return 1
    else:
        return 0


def collision_with_self(snake_position):
    snake_head = snake_position[0]
    if snake_head in snake_position[1:]:
        return 1
    else:
        return 0


class SnekEnv(gym.Env):
    metadata = {"render_modes": ["human"], "render_fps": 30}

    def __init__(self):
        super(SnekEnv, self).__init__()
        # Define action and observation space
        # They must be gym.spaces objects
        # Example when using discrete actions:
        self.action_space = spaces.Discrete(4)
        # Example for using image as input (channel-first; channel-last also works):
        self.observation_space = spaces.Box(low=0, high=1, shape=(8,), dtype=np.float32)

    def step(self, action):
        return None, None, None, None, None

    def reset(self, seed=None, options=None):
        self.img = np.zeros((500, 500, 3), dtype="uint8")

        # Initial Snake and Apple position
        self.snake_position = [[250, 250], [240, 250], [230, 250]]
        self.apple_position = [
            random.randrange(1, 50) * 10,
            random.randrange(1, 50) * 10,
        ]

        self.score = 0
        self.snake_head = self.snake_position[0].copy()

        self.done = False
        self.truncated = False

        observation = self.constructObservation()

        return observation, {}  # observation, info

    def render(self, mode="human"):
        # vvvv    HERE HERE HERE    vvvv
        cv2.imshow("Snake", self.img)  # Show the current frame
        cv2.waitKey(
            50
        )  # wait 50ms. Otherwise a person can't see anything because it is too fast

        # Create the next frame to be rendered
        self.img = np.zeros((500, 500, 3), dtype="uint8")  # create empty board

        # Display Apple
        cv2.rectangle(
            self.img,
            (self.apple_position[0], self.apple_position[1]),
            (self.apple_position[0] + 10, self.apple_position[1] + 10),
            (0, 0, 255),
            3,
        )
        # Display Snake
        for position in self.snake_position:
            cv2.rectangle(
                self.img,
                (position[0], position[1]),
                (position[0] + 10, position[1] + 10),
                (0, 255, 0),
                3,
            )
        # ^^^^    HERE HERE HERE ^^^^

    def close(self):
        pass

    def constructObservation(self):
        # Used to create a current observation
        """Observation holds: (all 1 or 0 values)
        * is apple to the left of head
        * is apple to the right of head
        * is apple above head
        * is apple below head
        * is wall or tail to the left of head
        * is wall or tail to the right of head
        * is wall or tail above head
        * is wall or tail below head
        """
        observation = []

        if self.apple_position[0] < self.snake_head[0]:  # to the left
            observation.extend([1, 0])
        else:
            observation.extend([0, 1])

        if self.apple_position[1] < self.snake_head[1]:  # above
            observation.extend([1, 0])
        else:
            observation.extend([0, 1])

        positions = self.snake_position.copy()
        head = positions[0].copy()

        # Check left
        head_left = [head[0] - 10, head[1]]
        positions[0] = head_left
        o = collision_with_boundaries(head_left) or collision_with_self(positions)
        observation.append(o)

        # Check right
        head_right = [head[0] + 10, head[1]]
        positions[0] = head_right
        o = collision_with_boundaries(head_right) or collision_with_self(positions)
        observation.append(o)

        # Check above
        head_above = [head[0], head[1] - 10]
        positions[0] = head_above
        o = collision_with_boundaries(head_above) or collision_with_self(positions)
        observation.append(o)

        # Check below
        head_below = [head[0], head[1] + 10]
        positions[0] = head_below
        o = collision_with_boundaries(head_below) or collision_with_self(positions)
        observation.append(o)

        observation = np.array(observation, dtype=np.float32)
        return observation


<div class="not_clean">

Nato bomo dodali `step()` metodo. Poleg observacije bomo tukaj definirali še naš reward.

Kot reward bomo začeli s preprosto enačbo - reward je enak dolžini kače.
</div>

In [ ]:
import random

import gymnasium as gym
import numpy as np
from gymnasium import spaces
import cv2


def collision_with_apple(apple_position, score):
    apple_position = [random.randrange(1, 50) * 10, random.randrange(1, 50) * 10]
    score += 1
    return apple_position, score


def collision_with_boundaries(snake_head):
    if (
        snake_head[0] >= 500
        or snake_head[0] < 0
        or snake_head[1] >= 500
        or snake_head[1] < 0
    ):
        return 1
    else:
        return 0


def collision_with_self(snake_position):
    snake_head = snake_position[0]
    if snake_head in snake_position[1:]:
        return 1
    else:
        return 0


class SnekEnv(gym.Env):
    metadata = {"render_modes": ["human"], "render_fps": 30}

    def __init__(self):
        super(SnekEnv, self).__init__()
        # Define action and observation space
        # They must be gym.spaces objects
        # Example when using discrete actions:
        self.action_space = spaces.Discrete(4)
        # Example for using image as input (channel-first; channel-last also works):
        self.observation_space = spaces.Box(low=0, high=1, shape=(8,), dtype=np.float32)
        # vvvv    HERE HERE HERE    vvvv
        self.render_training = False
        # ^^^^    HERE HERE HERE    ^^^^

    def step(self, action):
        # vvvv    HERE HERE HERE    vvvv
        self.render() if self.render_training else None  # If i want to see training or not

        # Change the head position based on the button direction
        if action == 1:  # move RIGHT
            self.snake_head[0] += 10
        elif action == 0:  # move LEFT
            self.snake_head[0] -= 10
        elif action == 2:  # move DOWN
            self.snake_head[1] += 10
        elif action == 3:  # move UP
            self.snake_head[1] -= 10

        # Calculating reward
        self.reward = 0

        # Increase Snake length on eating apple
        if self.snake_head == self.apple_position:
            self.apple_position, self.score = collision_with_apple(
                self.apple_position, self.score
            )
            self.snake_position.insert(0, list(self.snake_head))
        else:
            self.snake_position.insert(0, list(self.snake_head))
            self.snake_position.pop()

        # On collision kill the snake and print the score
        if (
            collision_with_boundaries(self.snake_head) == 1
            or collision_with_self(self.snake_position) == 1
        ):
            font = cv2.FONT_HERSHEY_SIMPLEX
            self.img = np.zeros((500, 500, 3), dtype="uint8")
            cv2.putText(
                self.img,
                "Your Score is {}".format(self.score),
                (140, 250),
                font,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            self.done = True  # if sneak hits itself it dies

        self.reward += len(self.snake_position)

        info = {}

        observation = self.constructObservation()

        return (
            observation,
            self.reward,
            self.done,
            self.truncated,
            info,
        )  # observation, reward, terminated, truncated, info
        # ^^^^    HERE HERE HERE    ^^^^

    def reset(self, seed=None, options=None):
        self.img = np.zeros((500, 500, 3), dtype="uint8")

        # Initial Snake and Apple position
        self.snake_position = [[250, 250], [240, 250], [230, 250]]
        self.apple_position = [
            random.randrange(1, 50) * 10,
            random.randrange(1, 50) * 10,
        ]

        self.score = 0
        self.snake_head = self.snake_position[0].copy()

        self.done = False
        self.truncated = False

        observation = self.constructObservation()

        return observation, {}  # observation, info

    def render(self, mode="human"):
        cv2.imshow("Snake", self.img)  # Show the current frame
        cv2.waitKey(
            50
        )  # wait 50ms. Otherwise a person can't see anything because it is too fast

        # Create the next frame to be rendered
        self.img = np.zeros((500, 500, 3), dtype="uint8")  # create empty board

        # Display Apple
        cv2.rectangle(
            self.img,
            (self.apple_position[0], self.apple_position[1]),
            (self.apple_position[0] + 10, self.apple_position[1] + 10),
            (0, 0, 255),
            3,
        )
        # Display Snake
        for position in self.snake_position:
            cv2.rectangle(
                self.img,
                (position[0], position[1]),
                (position[0] + 10, position[1] + 10),
                (0, 255, 0),
                3,
            )

    def close(self):
        pass

    def constructObservation(self):
        # Used to create a current observation
        """Observation holds: (all 1 or 0 values)
        * is apple to the left of head
        * is apple to the right of head
        * is apple above head
        * is apple below head
        * is wall or tail to the left of head
        * is wall or tail to the right of head
        * is wall or tail above head
        * is wall or tail below head
        """
        observation = []

        if self.apple_position[0] < self.snake_head[0]:  # to the left
            observation.extend([1, 0])
        else:
            observation.extend([0, 1])

        if self.apple_position[1] < self.snake_head[1]:  # above
            observation.extend([1, 0])
        else:
            observation.extend([0, 1])

        positions = self.snake_position.copy()
        head = positions[0].copy()

        # Check left
        head_left = [head[0] - 10, head[1]]
        positions[0] = head_left
        o = collision_with_boundaries(head_left) or collision_with_self(positions)
        observation.append(o)

        # Check right
        head_right = [head[0] + 10, head[1]]
        positions[0] = head_right
        o = collision_with_boundaries(head_right) or collision_with_self(positions)
        observation.append(o)

        # Check above
        head_above = [head[0], head[1] - 10]
        positions[0] = head_above
        o = collision_with_boundaries(head_above) or collision_with_self(positions)
        observation.append(o)

        # Check below
        head_below = [head[0], head[1] + 10]
        positions[0] = head_below
        o = collision_with_boundaries(head_below) or collision_with_self(positions)
        observation.append(o)

        observation = np.array(observation, dtype=np.float32)
        return observation


<div class="not_clean">

`close()` metodo bomo tudi izpustili, saj ne potrebujemo narediti nobenga posebnega zaključka.
</div>

<div class="not_clean">

Stable Defussion knjiznjica nam ponudi tudi metodo, s katero lahko okvirno potestiramo naše okolje.
</div>

In [ ]:
from stable_baselines3.common.env_checker import check_env
from snakeenv import SnekEnv


env = SnekEnv()
# It will check your custom environment and output additional warnings if needed
check_env(env, warn=True)

<div class="not_clean">

Če ne dobimo nobene napake lahko nadaljujemo z učenjem našega modela.

Prvo si bomo pogledali agenta, ki vzema random akcije:
</div>

In [ ]:
from snakeenv import SnekEnv

env = SnekEnv()

episodes = 5
for ep in range(episodes):
    env.reset()
    total_reward = 0
    steps = 0

    done = False
    while not done:
        env.render()
        observation, reward, done, _, info = env.step(env.action_space.sample())

        total_reward += reward
        steps += 1

    env.render()
    print("Steps taken", steps)
    print("Total reward", total_reward)


<div class="not_clean">

Sedaj pa lahko treniramo agenta. Uporabili bomo PPO algoritm.
</div>

In [ ]:
from stable_baselines3 import PPO
from snakeenv import SnekEnv

env = SnekEnv()
policy = "MlpPolicy"
model_name = "snek"

try:
    model = PPO.load(f"./models/{model_name}", env)
except FileNotFoundError as e:
    model = PPO(policy, env)

model.learn(3_000, progress_bar=True)
model.save(f"models/{model_name}")

done = False
obs, _ = env.reset()
while not done:
    env.render()
    action, _state = model.predict(obs, deterministic=True)
    obs, reward, done, truncated, info = env.step(action)
    if truncated:
        done = True
env.render()

<div class="not_clean">

Agenta smo trenirali zelo malo časa, zato se ni veliko naučil. 
    
Za napraj nam bo tudi koristilo imeti metodo, s katero lahko prikažemo agentovo igranje tekom učenja.
</div>

In [ ]:
from stable_baselines3 import PPO
from snakeenv import SnekEnv

env = SnekEnv()
policy = "MlpPolicy"
model_name = "snek_V1"
env.render_training = False
train = True

try:
    model = PPO.load(f"./models/{model_name}", env)
except FileNotFoundError as e:
    model = PPO(policy, env)

model.learn(3_000, progress_bar=True) if train else None
model.save(f"models/{model_name}") if train else None

done = False
obs, _ = env.reset()
while not done:
    env.render()

    action, _state = model.predict(obs, deterministic=True)
    obs, reward, done, truncated, info = env.step(action)
    if truncated:
        done = True

<div class="not_clean">

Sedaj bi sledilo učenje in igranje s tem kaj naš agent "vidi" (**observation**) in kakšno nagrado prejme kdaj (**reward function**).
    
</div>

<div class="not_clean">

Moje učenje je bilo sledeče:

Začeli sm s to postavitvijo kot jo imamo. Učili smo 100_000 korakov.
    
Kača se je naučila, da naj se ne ubije. Naučila se je vrteti v krogu.
    
Problem je, ker kača dobi večjo nagrado šele, ko ponesreči pobere jabolko. Ker pa je plošča velika se to zgodi zelo malokrat.
    
(Model: `snek_V1`)
</div>

<div class="not_clean">

2. Spremenil sem reward funkcijo tako, da se je dodalo veliko negativno nagrado (-1_000), če je kača umrla. Če je kača pojedla jabolk, je dobila veliko nagrado (100). Če se je kača premaknila proti jabolku dobi +1.

Treniralo se je za 100_000 korakov.
    
Problem je, ker je trajanje episode neskončno. Kača se lahko vrti v krogu in vsake toliko dobi točke samo zato ker se je ponesreči približala jabolku.

(Model: `snek_V2`)
</div>

<div class="not_clean">

3. Definiralo se je maximalno trajanje episode. Episoda se konča po 200 korakih. Če se episoda tako konča, kača ne dobi nobene negativne nagrade.
    
Treniralo se je dodatnih 100_000 korakov.
    
Kača se je sedaj naučila obrniti proti jabolku in ga pojesti. Pot ni optimizirana ampak kača dela nepotrebne vijuge. Problem je, ker je nagrajena za to, da se približa jabolku, če pa se oddalji pa ne prejme nagrade.

(Model: `snek_V3`)
</div>

<div class="not_clean">

4. Nagrado se je spremenilo tako, da kača sedaj prejme -1, če se je oddaljila od jabolka.
    
Treniralo se je še ene 100_000 korakov.
    
Kača sedaj obere dokaj direktno pot do jabolka in uspešno pobira nova jabolka.

(Model: `snek_V4`)
</div>

In [ ]:
import random

import gymnasium as gym
import numpy as np
from gymnasium import spaces
import cv2

EP_LENGTH = 300


def collision_with_apple(apple_position, score):
    apple_position = [random.randrange(1, 50) * 10, random.randrange(1, 50) * 10]
    score += 1
    return apple_position, score


def collision_with_boundaries(snake_head):
    if (
        snake_head[0] >= 500
        or snake_head[0] < 0
        or snake_head[1] >= 500
        or snake_head[1] < 0
    ):
        return 1
    else:
        return 0


def collision_with_self(snake_position):
    snake_head = snake_position[0]
    if snake_head in snake_position[1:]:
        return 1
    else:
        return 0


class SnekEnv(gym.Env):
    metadata = {"render_modes": ["human"], "render_fps": 30}

    def __init__(self):
        super(SnekEnv, self).__init__()
        # Define action and observation space
        # They must be gym.spaces objects
        # Example when using discrete actions:
        self.action_space = spaces.Discrete(4)
        # Example for using image as input (channel-first; channel-last also works):
        self.observation_space = spaces.Box(low=0, high=1, shape=(8,), dtype=np.float32)
        self.render_training = False

    def step(self, action):
        self.render() if self.render_training else None  # If i want to see training or not

        # Change the head position based on the button direction
        if action == 1:  # move RIGHT
            self.snake_head[0] += 10
        elif action == 0:  # move LEFT
            self.snake_head[0] -= 10
        elif action == 2:  # move DOWN
            self.snake_head[1] += 10
        elif action == 3:  # move UP
            self.snake_head[1] -= 10
        self.ep_steps += 1

        # Calculating reward
        self.reward = 0

        prev_apple_dist = np.sqrt(
            (self.apple_position[0] - self.snake_position[1][0]) ** 2
            + (self.apple_position[1] - self.snake_position[1][1]) ** 2
        )
        head_apple_dist = np.sqrt(
            (self.apple_position[0] - self.snake_head[0]) ** 2
            + (self.apple_position[1] - self.snake_head[1]) ** 2
        )  # current head distance to apple

        if head_apple_dist < prev_apple_dist:
            self.reward += 1
        else:
            self.reward -= 1

        # Increase Snake length on eating apple
        if self.snake_head == self.apple_position:
            self.apple_position, self.score = collision_with_apple(
                self.apple_position, self.score
            )
            self.snake_position.insert(0, list(self.snake_head))
            self.reward += 100
        else:
            self.snake_position.insert(0, list(self.snake_head))
            self.snake_position.pop()

        # On collision kill the snake and print the score
        if (
            collision_with_boundaries(self.snake_head) == 1
            or collision_with_self(self.snake_position) == 1
        ):
            font = cv2.FONT_HERSHEY_SIMPLEX
            self.img = np.zeros((500, 500, 3), dtype="uint8")
            cv2.putText(
                self.img,
                "Your Score is {}".format(self.score),
                (140, 250),
                font,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            self.done = True  # if sneak hits itself it dies
            self.reward -= 1_000
            print("Suicide")

        self.reward += len(self.snake_position)

        if self.ep_steps >= EP_LENGTH:
            self.truncated = True

        info = {}

        observation = self.constructObservation()

        return (
            observation,
            self.reward,
            self.done,
            self.truncated,
            info,
        )  # observation, reward, terminated, truncated, info

    def reset(self, seed=None, options=None):
        self.ep_steps = 0  # how many steps we made since last reset
        self.img = np.zeros((500, 500, 3), dtype="uint8")

        # Initial Snake and Apple position
        self.snake_position = [[250, 250], [240, 250], [230, 250]]
        self.apple_position = [
            random.randrange(1, 50) * 10,
            random.randrange(1, 50) * 10,
        ]

        self.score = 0
        self.snake_head = self.snake_position[0].copy()

        self.done = False
        self.truncated = False

        observation = self.constructObservation()

        return observation, {}  # observation, info

    def render(self, mode="human"):
        cv2.imshow("Snake", self.img)  # Show the current frame
        cv2.waitKey(
            50
        )  # wait 50ms. Otherwise a person can't see anything because it is too fast

        # Create the next frame to be rendered
        self.img = np.zeros((500, 500, 3), dtype="uint8")  # create empty board

        # Display Apple
        cv2.rectangle(
            self.img,
            (self.apple_position[0], self.apple_position[1]),
            (self.apple_position[0] + 10, self.apple_position[1] + 10),
            (0, 0, 255),
            3,
        )
        # Display Snake
        for position in self.snake_position:
            cv2.rectangle(
                self.img,
                (position[0], position[1]),
                (position[0] + 10, position[1] + 10),
                (0, 255, 0),
                3,
            )

    def close(self):
        pass

    def constructObservation(self):
        # Used to create a current observation
        """Observation holds: (all 1 or 0 values)
        * is apple to the left of head
        * is apple to the right of head
        * is apple above head
        * is apple below head
        * is wall or tail to the left of head
        * is wall or tail to the right of head
        * is wall or tail above head
        * is wall or tail below head
        """
        observation = []

        if self.apple_position[0] < self.snake_head[0]:  # to the left
            observation.extend([1, 0])
        else:
            observation.extend([0, 1])

        if self.apple_position[1] < self.snake_head[1]:  # above
            observation.extend([1, 0])
        else:
            observation.extend([0, 1])

        positions = self.snake_position.copy()
        head = positions[0].copy()

        # Check left
        head_left = [head[0] - 10, head[1]]
        positions[0] = head_left
        o = collision_with_boundaries(head_left) or collision_with_self(positions)
        observation.append(o)

        # Check right
        head_right = [head[0] + 10, head[1]]
        positions[0] = head_right
        o = collision_with_boundaries(head_right) or collision_with_self(positions)
        observation.append(o)

        # Check above
        head_above = [head[0], head[1] - 10]
        positions[0] = head_above
        o = collision_with_boundaries(head_above) or collision_with_self(positions)
        observation.append(o)

        # Check below
        head_below = [head[0], head[1] + 10]
        positions[0] = head_below
        o = collision_with_boundaries(head_below) or collision_with_self(positions)
        observation.append(o)

        observation = np.array(observation, dtype=np.float32)
        return observation
